In [1]:
%%time
import xarray as xr
import numpy as np
import joblib as jb
import time
import pandas as pd
import os, glob

CPU times: user 784 ms, sys: 120 ms, total: 904 ms
Wall time: 2.88 s


In [ ]:
os.chdir('/g/data/xv83/jp6794/NeurIPS23/ML_stuff/Harish')

In [3]:
import dask.distributed
import os
import tempfile

_dask_client = None
_tmpdir = None
def Client(threads=1, malloc_trim_threshold=None):
    global _dask_client, _tmpdir

    env = {}

    if malloc_trim_threshold is not None:
        env["MALLOC_TRIM_THRESHOLD_"] = str(
            dask.utils.parse_bytes(malloc_trim_threshold)
        )

    if _dask_client is None:
        _tmpdir = tempfile.TemporaryDirectory("dask-worker-space")
        workers = int(os.environ["PBS_NCPUS"]) // threads
        _dask_client = dask.distributed.Client(
            n_workers=workers,
            threads_per_worker=threads,
            memory_limit=int(os.environ["PBS_VMEM"]) / (workers),
            local_directory=_tmpdir.name,
            env=env,
            )
    return _dask_client
Client()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 252.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42815,Workers: 28
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 252.00 GiB
Comm: tcp://127.0.0.1:32781,Total threads: 1
Dashboard: /proxy/38513/status,Memory: 9.00 GiB
Nanny: tcp://127.0.0.1:44417,


In [65]:
# Function to find the closest index in a 1D array
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx
def era5_sfc_hourly(year,par,par_name,target_lat,target_lon):
    folder = "/g/data/rt52/era5/single-levels/reanalysis"
    file = f'{folder}/{par}/{year}/{par}_era5_oper_sfc_{year}'+'*.nc'
    hr_data = xr.open_mfdataset(file,
                                chunks={'lat': -1, 'lon': -1, 'time': 24}, 
                                parallel=True)
    latitude = hr_data.latitude
    longitude = hr_data.longitude
    lat_idx = find_nearest(latitude, target_lat)
    lon_idx = find_nearest(longitude, target_lon)
    hr_cor_data = hr_data[par_name].sel(latitude=latitude[lat_idx], 
                      longitude=longitude[lon_idx], method='nearest')
    return hr_cor_data

def era5_pres_hourly(year, par, level, target_lat,target_lon):
    folder = "/g/data/rt52/era5/pressure-levels/reanalysis/"
    file = folder+par+'/'+str(year)+'/'+par+'_era5_oper_pl_'+str(year)+'*.nc'
    hr_data = xr.open_mfdataset(file,
                                chunks={'level': 1,'lat': -1, 'lon': -1, 'time': 24}, 
                                parallel=True)
    latitude = hr_data.latitude
    longitude = hr_data.longitude
    lat_idx = find_nearest(latitude, target_lat)
    lon_idx = find_nearest(longitude, target_lon)
    hr_cor_data = hr_data[par].sel(level=level).sel(latitude=latitude[lat_idx], 
                      longitude=longitude[lon_idx], method='nearest').drop('level')
    hr_cor_data = hr_cor_data.rename(f'{par}_{level}')
    return hr_cor_data

### Extracting surface variables

In [ ]:
%%time
era5_dir = '/g/data/rt52/era5/single-levels/reanalysis'
pars = ['10u', '10v','100u', '100v','zust','i10fg',
       '2t','skt','stl1','2d','msl','blh','cbh',
       'ishf','ie','tcc','lcc','cape','cin','bld']
par_names = ['u10', 'v10', 'u100', 'v100','zust','i10fg',
            't2m','skt','stl1','d2m','msl','blh','cbh',
            'ishf','ie','tcc','lcc','cape','cin','bld']
lats=[54.0143,
        51.97,
        56.44050,
        51.706898,
        51.6463,
        51.9171,
        51.9990,
        52.306583,
        53.71250,
        54,
        54.177256]
lons=[6.58385,
        4.93,
        8.15080,
        3.034618,
        2.95141,
        3.6670,
        3.2760,
        4.008972,
        7.1522,
        6.623333,
        7.891971
        ]
data_dir = 'data'
for par,par_name in zip(pars,par_names):
    # create a folder with name par inside data_dir
    par_dir = f'{data_dir}/{par}'
    os.makedirs(par_dir, exist_ok=True)
    # --- extract data at each year ---#
    for year in np.arange(2000,2018+1):
        datasets = []
        for loc in range(len(lats)):
            ds = era5_sfc_hourly(year, par, par_name, lats[loc], lons[loc])
            datasets.append(ds.compute())
            del(ds)
        # Concatenate datasets along a new dimension ('location')
        combined_dataset = xr.concat(datasets, dim='location')
        combined_dataset['year'] = year
        file_path = f'{par_dir}/{year}.nc'
        if os.path.exists(file_path):
            os.remove(file_path)
        combined_dataset.to_netcdf(file_path)
        del(combined_dataset)
        print(par,year)
    # --- combining all years data ---#
    ds = xr.open_mfdataset(f'{data_dir}/{par}/*.nc',chunks={'lat': -1, 'lon': -1, 'time': -1}, 
                                parallel=True)
    file_path = f'{data_dir}/{par}.nc'
    if os.path.exists(file_path):
        os.remove(file_path)
    ds.to_netcdf(file_path)
    print(par)

### Extracting pressure level variables

In [ ]:
data_dir = 'data'
lvls = [975,950]
pars = ['u','v','t']
lats=[54.0143,
        51.97,
        56.44050,
        51.706898,
        51.6463,
        51.9171,
        51.9990,
        52.306583,
        53.71250,
        54,
        54.177256]
lons=[6.58385,
        4.93,
        8.15080,
        3.034618,
        2.95141,
        3.6670,
        3.2760,
        4.008972,
        7.1522,
        6.623333,
        7.891971
        ]
for par in pars:
    for level in lvls:
        # create a folder with name par and level inside data_dir
        par_dir = f'{data_dir}/{par}_{level}'
        os.makedirs(par_dir, exist_ok=True)
        # --- extract data at each year ---#
        for year in np.arange(2000,2018+1):
            datasets = []
            for loc in range(len(lats)):
                ds = era5_pres_hourly(year, par,level, lats[loc], lons[loc])
                datasets.append(ds.compute())
                del(ds)
            # Concatenate datasets along a new dimension ('location')
            combined_dataset = xr.concat(datasets, dim='location')
            combined_dataset['year'] = year
            file_path = f'{par_dir}/{year}.nc'
            if os.path.exists(file_path):
                os.remove(file_path)
            combined_dataset.to_netcdf(file_path)
            del(combined_dataset)
            print(par,level,year)
        # --- combining all years data ---#
        ds = xr.open_mfdataset(f'{par_dir}/*.nc',chunks={'lat': -1, 'lon': -1, 'time': -1}, 
                                    parallel=True)
        file_path = f'{data_dir}/{par}_{level}.nc'
        if os.path.exists(file_path):
            os.remove(file_path)
        ds.to_netcdf(file_path)
        print(par)

## Combining all variables into one file

In [31]:
client.close()
cluster.close()

In [3]:
import dask.distributed as dd
import dask
import dask.array as da
# Create a Dask cluster
print("Starting parallel computing...")
cluster = dd.LocalCluster(n_workers=16, dashboard_address=':22722')

# Connect to the cluster
client = dd.Client(cluster)
client

Starting parallel computing...


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:22722/status,
Dashboard: http://127.0.0.1:22722/status,Workers: 16
Total threads: 96,Total memory: 187.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34271,Workers: 16
Dashboard: http://127.0.0.1:22722/status,Total threads: 96
Started: Just now,Total memory: 187.55 GiB
Comm: tcp://127.0.0.1:40411,Total threads: 6
Dashboard: http://127.0.0.1:37037/status,Memory: 11.72 GiB
Nanny: tcp://127.0.0.1:46183,


In [15]:
def compute_wind_speed(par1,par_name1,par2,par_name2, par_name):
    ds1 = xr.open_dataset(f'ERA5_complete/{par1}.nc',chunks={'lat': -1, 'lon': -1, 'time': -1}).compute()
    ds2 = xr.open_dataset(f'ERA5_complete/{par2}.nc',chunks={'lat': -1, 'lon': -1, 'time': -1}).compute()  
    ds = (ds1[par_name1]**2+ds2[par_name2]**2)**0.5
    ds = ds.rename(par_name)
    return ds
def compute_alpha(dataset, par_name1,par_name2, par_name):
    ds = np.log(dataset[par_name2]/dataset[par_name1])/np.log(100/10)
    ds = ds.rename(par_name)
    return ds
def compute_gradient(dataset, par_name1,par_name2, par_name):
    ds = dataset[par_name2]-dataset[par_name1]
    ds = ds.rename(par_name)
    return ds

In [9]:
%%time
combined_dataset = xr.Dataset()
# --- 10m wind ---#
ds = compute_wind_speed('10u','u10','10v','v10', '10ws')
combined_dataset = xr.merge([combined_dataset, ds])

# --- 100m wind ---#
ds = compute_wind_speed('100u','u100','100v','v100', '100ws')
combined_dataset = xr.merge([combined_dataset, ds])

# --- 975 wind ---#
ds = compute_wind_speed('u_975','u_975','v_975','v_975', '975ws')
combined_dataset = xr.merge([combined_dataset, ds])

# --- 950 wind ---#
ds = compute_wind_speed('u_950','u_950','v_950','v_950', '950ws')
combined_dataset = xr.merge([combined_dataset, ds])

pars = ['zust','i10fg',
       '2t','skt','stl1','2d','msl','blh','cbh',
       'ishf','ie','tcc','lcc','cape','cin','bld','t_975','t_950']
par_names = ['zust','i10fg',
            't2m','skt','stl1','d2m','msl','blh','cbh',
            'ishf','ie','tcc','lcc','cape','cin','bld','t_975','t_950']

for par in pars:
    file_path = f'ERA5_complete/{par}.nc'
    ds = xr.open_dataset(file_path,chunks={'lat': -1, 'lon': -1, 'time': -1}).compute()
    combined_dataset = xr.merge([combined_dataset, ds])
    print(par)

# === derived parameters === #
# --- 100 alpha ---#
ds = compute_alpha(combined_dataset,'10ws','100ws','100alpha')
combined_dataset = xr.merge([combined_dataset, ds])

# --- 975 wind gradient ---#
ds = compute_gradient(combined_dataset,'100ws','975ws','975wsgrad')
combined_dataset = xr.merge([combined_dataset, ds])

# --- 950 wind gradient ---#
ds = compute_gradient(combined_dataset,'975ws','950ws','950wsgrad')
combined_dataset = xr.merge([combined_dataset, ds]) 

# --- 2m temperature gradient ---#
ds = compute_gradient(combined_dataset,'skt','t2m','2mtempgrad')
combined_dataset = xr.merge([combined_dataset, ds]) 

# --- skin temperature gradient ---#
ds = compute_gradient(combined_dataset,'stl1','skt','sktempgrad')
combined_dataset = xr.merge([combined_dataset, ds]) 

# --- Temperature dewpoint spread ---#
ds = compute_gradient(combined_dataset,'d2m','t2m','dewtempsprd')
combined_dataset = xr.merge([combined_dataset, ds]) 
    
# --- 950 temperature gradient ---#
ds = compute_gradient(combined_dataset,'t_975','t_950','950tempgrad')
combined_dataset = xr.merge([combined_dataset, ds]) 

# --- 975 temperature gradient ---#
ds = compute_gradient(combined_dataset,'t2m','t_975','975tempgrad')
combined_dataset = xr.merge([combined_dataset, ds]) 

# === save file ===#
file_path = f'ERA5.nc'
if os.path.exists(file_path):
    os.remove(file_path)
combined_dataset.to_netcdf(file_path)

zust
i10fg
2t
skt
stl1
2d
msl
blh
cbh
ishf
ie
tcc
lcc
cape
cin
bld
t_975
t_950
CPU times: user 2min 2s, sys: 25.8 s, total: 2min 28s
Wall time: 5min 8s


In [57]:
ERA5 = xr.open_dataset('ERA5.nc')
combined_dataset = ERA5
HR =  pd.to_datetime(ERA5.time).hour
# --- sin converted hour ---#
ds = xr.DataArray(np.sin(2 * np.pi / 24 * HR), coords=[ERA5.time], dims=['time'], name='sinHR')
combined_dataset = xr.merge([combined_dataset, ds])

# --- cosine converted hour ---#
ds = xr.DataArray(np.cos(2 * np.pi / 24 * HR), coords=[ERA5.time], dims=['time'], name='cosHR')
combined_dataset = xr.merge([combined_dataset, ds])

JDAY  = pd.to_datetime(ERA5.time).dayofyear
# --- sin converted day ---#
ds = xr.DataArray(np.sin(2 * np.pi / 366 * JDAY), coords=[ERA5.time], dims=['time'], name='sinJDAY')
combined_dataset = xr.merge([combined_dataset, ds])

# --- cosine converted day ---#
ds = xr.DataArray(np.cos(2 * np.pi / 366 * JDAY), coords=[ERA5.time], dims=['time'], name='cosJDAY')
combined_dataset = xr.merge([combined_dataset, ds])

In [60]:
# === save file ===#
file_path = f'ERA5.nc'
if os.path.exists(file_path):
    os.remove(file_path)
combined_dataset.to_netcdf(file_path)

In [19]:
def compute_temp_shift(dataset, par_name, delta_t):
    ds = dataset[par_name]-dataset[par_name].shift(time=delta_t)
    ds = ds.rename(f'{par_name}_delta{delta_t}')
    return ds

In [27]:
ERA5 = xr.open_dataset('ERA5.nc')
combined_dataset = ERA5
pars = ['10ws','100ws','975ws','950ws']
for par in pars:
    for delta_t in range(1,7):
        ds = compute_temp_shift(combined_dataset,par,delta_t)
        combined_dataset = xr.merge([combined_dataset, ds])
        print(ds.name)

10ws_delta1
10ws_delta2
10ws_delta3
10ws_delta4
10ws_delta5
10ws_delta6
100ws_delta1
100ws_delta2
100ws_delta3
100ws_delta4
100ws_delta5
100ws_delta6
975ws_delta1
975ws_delta2
975ws_delta3
975ws_delta4
975ws_delta5
975ws_delta6
950ws_delta1
950ws_delta2
950ws_delta3
950ws_delta4
950ws_delta5
950ws_delta6


In [29]:
# === save file ===#
file_path = f'ERA5.nc'
if os.path.exists(file_path):
    os.remove(file_path)
combined_dataset.to_netcdf(file_path)

In [30]:
xr.open_dataset('ERA5.nc')

<xarray.Dataset>
Dimensions:       (location: 11, time: 166560)
Coordinates:
    longitude     (location) float32 ...
    latitude      (location) float32 ...
  * time          (time) datetime64[ns] 2000-01-01 ... 2018-12-31T23:00:00
    year          (time) int64 ...
Dimensions without coordinates: location
Data variables: (12/58)
    10ws          (location, time) float32 ...
    100ws         (location, time) float32 ...
    975ws         (location, time) float32 ...
    950ws         (location, time) float32 ...
    zust          (location, time) float32 ...
    i10fg         (location, time) float32 ...
    ...            ...
    950ws_delta1  (location, time) float32 ...
    950ws_delta2  (location, time) float32 ...
    950ws_delta3  (location, time) float32 ...
    950ws_delta4  (location, time) float32 ...
    950ws_delta5  (location, time) float32 ...
    950ws_delta6  (location, time) float32 ...